In [1]:
import initialize_notebook

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

import io
from ams import config
from sklearn.preprocessing import StandardScaler
import pandas_datareader.data as web
import math
import os
import datetime as dt
import sys
from pathlib import Path
from typing import Dict

from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, Row, StructField, DoubleType, ArrayType, StringType
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from ams.config import constants
from ams.services import file_services
from ams.services import spark_service
from ams.utils import sentiment
from pyspark.storagelevel import StorageLevel

%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\ProgramData\Miniconda3\envs\alpha_media_signal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


Setting up logging...
Will use logging path: C:\Users\Chris\workspaces\data\logs\alpha_media_signal


In [2]:
# import findspark

# findspark.init()
spark = spark_service.get_or_create('learning_prep')
spark.stop()
spark = spark_service.get_or_create('learning_prep')

In [3]:
twitter_folder = 'twitter'

tweet_sent_folders = [
    "tweet_sent_2020-09-14_01-54-46-424.27", "tweet_sent_2020-09-16_04-20-22-347.55", 
    "tweet_sent_2020-09-18_13-56-57-859.2", "tweet_sent_2020-09-18_22-23-53-311.06", 
    "tweet_sent_2020-09-20_12-55-15-934.3", "tweet_sent_2020-09-26_09-31-41-445.65", 
    "tweet_sent_2020-09-26_14-37-22-544.29", "tweet_sent_2020-09-28_21-45-23-139.81"
]

In [4]:
%%time

senti_dataframes = []

for t in tweet_sent_folders:
    file_path = Path(constants.DATA_PATH, twitter_folder, "sent_drop", t)
    df = spark.read.csv(str(file_path), header=True)
    print(f'df_1: {df.count()}')
    senti_dataframes.append(df) 

df_1: 2606202
df_1: 312411
df_1: 1376331
df_1: 690703
df_1: 909703
df_1: 623898
df_1: 463726
df_1: 724522
Wall time: 47.5 s


In [5]:
%%time

from functools import reduce
from pyspark.sql import DataFrame

df = reduce(DataFrame.unionByName, senti_dataframes)
senti_dataframes = None

df_red = df
# df.persist(StorageLevel.DISK_ONLY)
df = None

Wall time: 99 ms


In [6]:
print(df_red.columns)

['entities_urls_1', 'text', 'entities_user_mentions_2', 'entities_urls_2', 'user_profile_background_tile', 'entities_urls_3', 'user_screen_name', 'favorite_count', 'in_reply_to_screen_name', 'user_verified', 'metadata_result_type', 'user_listed_count', 'user_time_zone', 'user_profile_banner_url', 'entities_urls_0', 'user_has_extended_profile', 'in_reply_to_status_id', 'user_profile_background_color', 'in_reply_to_user_id', 'user_statuses_count', 'user_url', 'user_protected', 'user_profile_background_image_url', 'user_is_translation_enabled', 'user_utc_offset', 'contributors', 'user_location', 'user_profile_text_color', 'source', 'lang', 'metadata_iso_language_code', 'entities_user_mentions_0', 'user_profile_link_color', 'user_geo_enabled', 'user_profile_image_url_https', 'user_favourites_count', 'place_country', 'place_name', 'possibly_sensitive', 'user_description', 'user_profile_sidebar_fill_color', 'user_friends_count', 'id', 'user_notifications', 'user_default_profile', 'retweeted'

In [7]:
%%time

from ams.utils import date_utils
from pyspark.sql import types as T
from datetime import datetime

twitter_format = date_utils.TWITTER_LONG_FORMAT 
def udf_wrapper(date_string: str):
    return int(datetime.strptime(date_string, twitter_format).timestamp())
        
parse_udf = F.udf(udf_wrapper, T.IntegerType())

print(f'Count: {df_red.count()}')

df_ts = df_red.withColumn('created_at_timestamp', parse_udf(F.col('created_at')))
# df_red.persist(StorageLevel.DISK_ONLY)
df_red = None

Count: 7707496
Wall time: 35.1 s


In [8]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from ams.utils import date_utils

df_winn = df_ts.select(*["created_at",
                 "created_at_timestamp",
                 "user_screen_name",
                 "favorite_count",
                 "in_reply_to_screen_name",
                 "user_verified",
                 "metadata_result_type",
                 "user_listed_count",
                 "user_time_zone",
                 "user_has_extended_profile",
                 "in_reply_to_status_id",
                 "user_statuses_count",
                 "user_protected",
                 "user_is_translation_enabled",
                 "user_location",
                 "lang",
                 "user_geo_enabled",
                 "place_country",
                 "place_name",
                 "possibly_sensitive",
                 "user_friends_count",
                 "retweet_count",
                 "user_follow_request_sent",
                 "user_followers_count",
                 "f22_ticker",
                 "f22_has_cashtag",
                 "f22_ticker_in_text",
                 "f22_num_other_tickers_in_tweet",
                 "f22_sentiment_pos",
                 "f22_sentiment_neu",
                 "f22_sentiment_neg",
                 "f22_sentiment_compound"])
df_ts.persist(StorageLevel.DISK_ONLY)

DataFrame[entities_urls_1: string, text: string, entities_user_mentions_2: string, entities_urls_2: string, user_profile_background_tile: string, entities_urls_3: string, user_screen_name: string, favorite_count: string, in_reply_to_screen_name: string, user_verified: string, metadata_result_type: string, user_listed_count: string, user_time_zone: string, user_profile_banner_url: string, entities_urls_0: string, user_has_extended_profile: string, in_reply_to_status_id: string, user_profile_background_color: string, in_reply_to_user_id: string, user_statuses_count: string, user_url: string, user_protected: string, user_profile_background_image_url: string, user_is_translation_enabled: string, user_utc_offset: string, contributors: string, user_location: string, user_profile_text_color: string, source: string, lang: string, metadata_iso_language_code: string, entities_user_mentions_0: string, user_profile_link_color: string, user_geo_enabled: string, user_profile_image_url_https: string,

In [9]:
df_has_time = df_winn.where(F.col('created_at_timestamp').isNotNull())
# df_winn.persist(StorageLevel.DISK_ONLY)
df_winn = None

In [20]:
%%time

from ams.utils import twitter_utils
from pyspark.sql import functions as F, types as T
from ams.utils.date_utils import TZ_AMERICA_NEW_YORK, STANDARD_DAY_FORMAT
import pytz

# df_dated = twitter_utils.create_date_column(df_has_time)

# df_dated = df_has_time.withColumn('date', date_utils.convert_timestamp_to_nyc_date_str(F.col('created_at_timestamp')))

def convert_to_date_string(utc_timestamp: int):
#     return date_utils.convert_timestamp_to_nyc_date_str(utc_timestamp=utc_timestamp)
    dt_utc = datetime.fromtimestamp(utc_timestamp)
    dt_nyc = dt_utc.astimezone(pytz.timezone(TZ_AMERICA_NEW_YORK))
    return dt_nyc.strftime(STANDARD_DAY_FORMAT)

parse_udf = F.udf(convert_to_date_string, T.StringType())

df_has_time = df_has_time.withColumn('date', parse_udf(F.col('created_at_timestamp')))
# df_dated.persist(StorageLevel.DISK_ONLY)
# df_has_time = None

print(df_has_time.count())

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:12538)
Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:12538)

In [16]:
%%time

df_sample = df_has_time.sample(fraction=.001)

print(df_sample.count())

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1200, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\ProgramData\Miniconda3\envs\alpha_media_signal\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1212, in send_command
    "Error while recei

Py4JError: An error occurred while calling o211.count

In [17]:
%%time

df_pd = df_sample.toPandas().sort_values('date', inplace=False)

day_groups = df_pd.groupby(df_pd['date'])['date'].count()

day_groups.plot(kind='bar',figsize=(10,5),legend=None)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:12538)
Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1193, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1197, in send_command
    "Error while sending", e, proto.ERROR_ON_SEND)
py4j.protocol.Py4JNetworkError: Error while sendi

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:12538)

In [18]:
%%time

df_time_converted = df_dated.withColumn('user_followers_count', F.col('user_followers_count').cast(T.IntegerType())).withColumn('f22_sentiment_compound', F.col('f22_sentiment_compound').cast(T.FloatType()))

df_has_scores = df_time_converted.where(F.col('user_followers_count') > 0).where(F.col('f22_sentiment_compound') != 0)

print(f'Has followers and compound score: {df_has_scores.count()}')

df_com_score = df_has_scores.withColumn('f22_compound_score', F.col('user_followers_count') * F.col('f22_sentiment_compound'))

NameError: name 'df_dated' is not defined

In [19]:
%%time

from ams.services import dataframe_services

tag = 'learning_prep_drop'

sent_drop_path = Path(constants.DATA_PATH, twitter_folder, tag)

dataframe_services.persist_dataframe_as_csv(df=df_com_score, output_drop_folder_path=sent_drop_path, prefix=tag, num_output_files=1)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:12538)
Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:12538)
Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\li

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:12538)
Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Chris\workspaces\open_source\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it


NameError: name 'df_com_score' is not defined